In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, quantile_transform, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

In [34]:
df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Predictive Modelling | MBAN 5110 U /Hackathon/Updated_XYZ_Sportswear_Orders_Dataset.csv')

In [35]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d.%m.%Y')
df['Requested Delivery Date'] = pd.to_datetime(df['Requested Delivery Date'], format='%d.%m.%Y')
df['Order Month'] = df['Order Date'].dt.month
df['Order Year'] = df['Order Date'].dt.year
df['Delivery Month'] = df['Requested Delivery Date'].dt.month
df['Delivery Year'] = df['Requested Delivery Date'].dt.year
monthly_orders = df.groupby(['Order Year', 'Order Month']).agg({'Value': 'sum'}).reset_index()
monthly_orders = monthly_orders.sort_values(by=['Order Year', 'Order Month'])

def month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'


df['Season'] = df['Order Month'].apply(month_to_season)


df['Lead Time'] = (df['Requested Delivery Date'] - df['Order Date']).dt.days

print(df[['Order Date', 'Requested Delivery Date', 'Lead Time']].head())

df['Lead Time'] = (df['Requested Delivery Date'] - df['Order Date']).dt.days
df['Value/Item'] = df['Value']/df['Items']


df['Season_Num'] = df['Season'].map({'Winter': 1, 'Spring': 2, 'Summer': 3, 'Fall': 4})


df['advanced_order'] = (df['Lead Time'] >= 30).astype(int)

df.head()

  Order Date Requested Delivery Date  Lead Time
0 2009-01-01              2009-02-18         48
1 2009-01-01              2009-02-10         40
2 2009-01-01              2009-01-17         16
3 2009-01-03              2009-01-28         25
4 2009-01-03              2009-01-18         15


,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,...,Value per Item,Order Month,Order Year,Delivery Month,Delivery Year,Season,Lead Time,Value/Item,Season_Num,advanced_order
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,...,33.733750,1,2009,2,2009,Winter,48,33.733750,1,1
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,...,85.170000,1,2009,2,2009,Winter,40,85.170000,1,1
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,...,8.727143,1,2009,1,2009,Winter,16,8.727143,1,0
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,...,41.863333,1,2009,1,2009,Winter,25,41.863333,1,0
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,...,30.666000,1,2009,1,2009,Winter,15,30.666000,1,0


In [36]:
df_copy = df.copy()

In [37]:
df_copy['advanced_order_amount'] = df_copy['Items'] * df_copy['advanced_order']

In [38]:
df_copy['urgent_order_amount'] = df_copy['Items'] - df_copy['advanced_order_amount']

In [39]:
df_copy.head()

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order Type,Customer Order Code,Value,Currency,Items,...,Order Year,Delivery Month,Delivery Year,Season,Lead Time,Value/Item,Season_Num,advanced_order,advanced_order_amount,urgent_order_amount
0,2009-01-01,2009-02-18,DE,PK003,Economy Parka,VO,COD00001,269.87,EUR,8,...,2009,2,2009,Winter,48,33.733750,1,1,8,0
1,2009-01-01,2009-02-10,FR,PK001,Advanced Parka,VO,COD00002,170.34,EUR,2,...,2009,2,2009,Winter,40,85.170000,1,1,2,0
2,2009-01-01,2009-01-17,ES,PK002,Premium Parka,VO,COD00003,61.09,EUR,7,...,2009,1,2009,Winter,16,8.727143,1,0,0,7
3,2009-01-03,2009-01-28,IT,PK003,Economy Parka,VO,COD00004,251.18,EUR,6,...,2009,1,2009,Winter,25,41.863333,1,0,0,6
4,2009-01-03,2009-01-18,DE,PK004,Basic Parka,VO,COD00005,153.33,EUR,5,...,2009,1,2009,Winter,15,30.666000,1,0,0,5


In [41]:
df_grouped = df_copy.groupby(['Order Year', 'Order Month']).agg({'advanced_order_amount': 'sum', 'urgent_order_amount': 'sum'}).reset_index()


In [42]:
df_grouped.head()

,Order Year,Order Month,advanced_order_amount,urgent_order_amount
0,2009,1,147,156
1,2009,2,104,186
2,2009,3,174,205
3,2009,4,243,126
4,2009,5,157,142


In [40]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Assuming df_table is your DataFrame

# Data Preparation
y_table = df_table['Demand'][1:].values
X_table = df_table[['Demand', 'Advance_Demand']][:-1].values  # Include additional features here
L = df_table['Advance_Demand'][1:].values

# Ridge Regression with Cross-Validated Grid Search for Lambda
parameters = {'alpha': np.logspace(-10, 10, 21)}  # Range of lambda values
ridge = Ridge()
clf = GridSearchCV(ridge, parameters, scoring='neg_mean_absolute_percentage_error', cv=5)
clf.fit(X_table, y_table)

# Best lambda value
best_lambda = clf.best_params_['alpha']

# Ridge Regression with the Best Lambda
ridge = Ridge(alpha=best_lambda)
ridge.fit(X_table, y_table)

# Coefficients
beta_opt = ridge.coef_

# Predictions
predictions = ridge.predict(X_table)

# Calculate MAPE
mape = np.mean(np.abs((y_table - predictions) / y_table)) * 100

print(f'Best Lambda: {best_lambda}')
print(f'Optimal beta (coefficients): {beta_opt}')
print(f'MAPE: {mape}%')

NameError: name 'df_table' is not defined